<a href="https://colab.research.google.com/github/filipemcbarros/modelos_classificadores_conciliacao/blob/main/BERTikal_classificacao_conciliacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bibliotecas utilizadas**

In [ ]:
import pandas as pd
from google.colab import drive
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report,roc_auc_score,roc_curve,accuracy_score,matthews_corrcoef,f1_score
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
!pip install unidecode
!pip install ftfy
!pip install transformers==4.2.2
!pip install pyreadr
!pip install git+https://github.com/felipemaiapolo/legalnlp
!pip install transformers torch scikit-learn pandas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## **Método para carregamento das localidades**

In [ ]:
dataset = pd.read_csv("/content/drive/My Drive/Colab Notebooks/base_processos/dataset_final.csv")
dataset

# Funções de Limpeza para Corpus Bruto

In [ ]:
# Remover linhas vazias
dataset = dataset.dropna(subset=['corpus_limpo'])

# Contando o número de palavras na coluna 'corpus_limpo'
dataset['corpus_limpo'] = dataset['corpus_limpo'].fillna('')
dataset['num_palavras_corpus'] = dataset['corpus_limpo'].str.split().apply(len)

# Limpeza das linhas com corpus que possuem menos que 30 palavras para eliminar textos de referências a anexos e sem conteúdo útil
dataset = dataset.drop(dataset[dataset['num_palavras_corpus'] < 30].index)
dataset

In [ ]:
from legalnlp.clean_functions import clean_bert

dataset['corpus_bruto'] = dataset['corpus_bruto'].apply(lambda x:clean_bert(x))
dataset

In [ ]:
import re

#texto caixa baixa
#remover aspas simples, aspas duplas, reticências, vírgula, underline
#remover quebras de linha por espaço em branco simples
def clean_text_extra_round(text):
    text = text.lower()
    text = re.sub('[‘’“”\…,°ºª§]', ' ', text)
    text = text.replace("'","")
    text = text.replace("_","")
    text = text.strip('\n')
    text = text.strip('\t')
    text = clean_roman_numbers(text)
    text = re.sub("\s+", ' ', text)
    text = text.replace("\\", ' ')
    return text

#substituir múltiplos espaços em branco por um único espaço em branco
def multiple_one_blank_spaces(text):
  text = re.sub(r'\s+', ' ', text)
  return text

#remove números
def is_digit(text):
  text = re.sub('\w*\d\w*', '', text)
  return text

#remove pontuações
def is_punctuation(text):
  text = re.sub(r'[^\w\s]', '', text)
  return text

#remove algarismos romanos
def clean_roman_numbers(text):
    pattern = r"\b(?=[mdclxvii])m{0,4}(cm|cd|d?c{0,3})(xc|xl|l?x{0,3})([ii]x|[ii]v|v?[ii]{0,3})\b\.?"
    return re.sub(pattern, ' ', text)

In [ ]:
dataset['corpus_bruto'] = dataset['corpus_bruto'].apply(clean_text_extra_round)
dataset['corpus_bruto'] = dataset['corpus_bruto'].apply(is_digit)
dataset['corpus_bruto'] = dataset['corpus_bruto'].apply(is_punctuation)
dataset['corpus_bruto'] = dataset['corpus_bruto'].apply(clean_roman_numbers)
dataset['corpus_bruto'] = dataset['corpus_bruto'].apply(multiple_one_blank_spaces)

In [ ]:
features = dataset['corpus_bruto']
classes = dataset['conciliado']
processos = dataset['num_processo']

conciliados = features[(dataset['conciliado'] == 1)]
nao_conciliados = features[(dataset['conciliado'] == 0)]

# **Modelo de Linguagem BERTikal**

In [ ]:
import torch
import torch.nn as nn
import torch.utils.data as tdata
import torch.optim as optim
import transformers
from torch.utils.data import DataLoader
from transformers import AutoModel, AutoTokenizer, AutoConfig
from transformers import BertForPreTraining, BertModel, BertTokenizer, BertForMaskedLM, BertForNextSentencePrediction, BertForQuestionAnswering

print(torch.cuda.is_available())

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

In [ ]:
from legalnlp.get_premodel import *
# Fazendo o download do modelo pre-treinado BERTikal e o seu tokenizador
get_premodel('bert')

In [ ]:
# Caso esteja usando o Google Colab, não esqueça de ligar a GPU para o ambiente de execução
import IPython
from IPython.display import Image
from IPython.display import clear_output

bert_model =  BertModel.from_pretrained('/content/BERTikal/').to(device)
bert_tokenizer = BertTokenizer.from_pretrained('/content/BERTikal/vocab.txt', do_lower_case=False)

clear_output()

# configuração do BERTikal
bert_model.config

tokenizer = bert_tokenizer
model = bert_model

In [ ]:
data_text = list(features)

In [ ]:
# Aplicando o bert_tokenizer em nosso dataset com um comprimento máximo de 512 tokens
encoded_inputs = bert_tokenizer(data_text, padding=True, truncation=True, max_length=512, return_tensors="pt")

#Agora temos nossos encoded_input em um dicionário com 3 chaves
encoded_inputs.keys()

In [ ]:
# Enviando os tensores para para a GPU
input_ids = encoded_inputs['input_ids'].to(device)

In [ ]:
from tqdm import tqdm

# Criando o nosso vetor de features
features = []

# Aplicando o modelo pré-treinado em cada frase e adicionando-o ao nosso vetor

for i in tqdm(range(len(data_text))):

    with torch.no_grad():

      last_hidden_states = bert_model(input_ids[i:(i+1)])[1].cpu().numpy().reshape(-1).tolist()

    features.append(last_hidden_states)

# Criando um numpy array com as features extraidas
features = np.array(features)

In [ ]:
df_features = pd.DataFrame(features)
features_label = pd.concat([df_features, dataset['conciliado']], axis = 1)
features_label.shape

In [ ]:
print(features_label)

In [ ]:
!pip install catboost
clear_output()

# **Definindo Conjunto de Treinamento e Testes**

In [ ]:
# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(features_label.drop(columns = ['conciliado']), features_label['conciliado'], random_state = 1234,test_size = 0.3)

# SVC

In [ ]:
from sklearn.svm import SVC

#parâmetros com valores decididos pela função GridSearchCV
svcGaussian = SVC(kernel='rbf', C=1000.0)
svcGaussian.fit(X_train,y_train)
target_svc_gaussian = svcGaussian.predict(X_test)

# **Avaliação do modelo SVC**

In [ ]:
print('############# Relatório de Classificação - Modelo SVM #############')
print()

print("Relatório de Classificação:\n", classification_report(y_test_carregado, target_svc_gaussian, digits=4))
# imprimir a acurácia do modelo
print("Acurácia: {:.4f}\n".format(accuracy_score(y_test_carregado, target_svc_gaussian)))
# imprimir a área sob da curva
print("AUC: {:.4f}\n".format(roc_auc_score(y_test_carregado, target_svc_gaussian)))
print("MCC:  {:.4f}\n".format(matthews_corrcoef(y_test_carregado, target_svc_gaussian)))
print("F1-Score:  {:.4f}\n".format(f1_score(y_test_carregado, target_svc_gaussian)))

In [ ]:
print('#### Matriz de Confusão - Modelo SVM ####')
print()
cf_matrix = confusion_matrix(y_test_carregado,target_svc_gaussian)
sns.heatmap(cf_matrix, annot=True, fmt='.0f')
plt.show()

# Regressão Logística

In [ ]:
from sklearn.linear_model import LogisticRegression

#parâmetros com valores decididos pela função GridSearchCV
lr = LogisticRegression(solver='lbfgs', C=10.0)
lr.fit(X_train,y_train)
target_lr = lr.predict(X_test)

# **Avaliação do modelo Regressão Logística**



In [ ]:
print('############# Relatório de Classificação - Modelo Regressão Logística #############')
print()

print("Relatório de Classificação:\n", classification_report(y_test_carregado, target_lr, digits=4))
# imprimir a acurácia do modelo
print("Acurácia: {:.4f}\n".format(accuracy_score(y_test_carregado, target_lr)))
# imprimir a área sob da curva
print("AUC: {:.4f}\n".format(roc_auc_score(y_test_carregado, target_lr)))
print("MCC:  {:.4f}\n".format(matthews_corrcoef(y_test_carregado, target_lr)))
print("F1-Score:  {:.4f}\n".format(f1_score(y_test_carregado, target_lr)))

In [ ]:
print('#### Matriz de Confusão - Modelo Regressão Logística ####')
print()
cf_matrix = confusion_matrix(y_test_carregado,target_lr)
sns.heatmap(cf_matrix, annot=True, fmt='.0f')
plt.show()

# Árvore de Decisão

In [ ]:
from sklearn.tree import DecisionTreeClassifier

#parâmetros com valores decididos pela função GridSearchCV
dtc = DecisionTreeClassifier(criterion='“entropy”')
dtc.fit(word2vec_train_vectors, y_train)
target_dtc = dtc.predict(word2vec_test_vectors)

# **Avaliação do modelo Árvore de Decisão**



In [ ]:
print('############# Relatório de Classificação - Modelo Árvore de Decisão #############')
print()

print("Relatório de Classificação:\n", classification_report(y_test_carregado, target_dtc, digits=4))
# imprimir a acurácia do modelo
print("Acurácia: {:.4f}\n".format(accuracy_score(y_test_carregado, target_dtc)))
# imprimir a área sob da curva
print("AUC: {:.4f}\n".format(roc_auc_score(y_test_carregado, target_dtc)))
print("MCC:  {:.4f}\n".format(matthews_corrcoef(y_test_carregado, target_dtc)))
print("F1-Score:  {:.4f}\n".format(f1_score(y_test_carregado, target_dtc)))

In [ ]:
print('#### Matriz de Confusão - Modelo Árvore de Decisão ####')
print()
cf_matrix = confusion_matrix(y_test_carregado,target_dtc)
sns.heatmap(cf_matrix, annot=True, fmt='.0f')
plt.show()

# XGBoost

In [ ]:
# Treinamento do modelo XGBoost
import xgboost as xgb

# Convert the data into DMatrix format, which is required by XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set the parameters for XGBoost
params = {
    'criterion':'squared_error',
    'max_depth': 7,
    'learning_rate': 0.3,
    'n_estimators': 100
}

# Train the Boosted Trees model
bst = xgb.train(params, dtrain)

# Make predictions on the test set
target_xgb = bst.predict(dtest)

# **Avaliação do modelo XGBoost**

In [ ]:
# Definir um limite de decisão (exemplo: 0.5)
limite_decisao = 0.5

# Converter as previsões em classes binárias
previsoes_binarias = [1 if predicao > limite_decisao else 0 for predicao in target_xgb]

# Converter os rótulos reais em classes binárias
rotulos_binarios = [1 if rotulo > limite_decisao else 0 for rotulo in y_test]

print('############# Relatório de Classificação - Modelo XGBoost #############')
print()

print("Relatório de Classificação:\n", classification_report(rotulos_binarios, previsoes_binarias, digits=4))

print("Acurácia: {:.4f}\n".format(accuracy_score(rotulos_binarios, previsoes_binarias)))
# imprimir a área sob da curva
print("AUC: {:.4f}\n".format(roc_auc_score(rotulos_binarios, previsoes_binarias)))
print("MCC:  {:.4f}\n".format(matthews_corrcoef(rotulos_binarios, previsoes_binarias)))
print("F1-Score:  {:.4f}\n".format(f1_score(rotulos_binarios, previsoes_binarias)))

In [ ]:
print('#### Matriz de Confusão - Modelo XGBoost ####')
print()
cf_matrix = confusion_matrix(y_test,previsoes_binarias)
sns.heatmap(cf_matrix, annot=True, fmt='.0f')
plt.show()